## フォロースルーの探し方

[くわしくはこちら ](https://www.evernote.com/shard/s6/nl/104429507/d6c94ed8-4eb2-4890-a274-d63ff33650ae?title=%E3%81%99%E3%81%94%E8%85%95%E6%8A%95%E8%B3%87%E5%AE%B6%E3%83%BBDUKE%E3%80%82%E3%81%95%E3%82%93%E3%81%AB%E8%81%9E%E3%81%8F%E3%80%8C%E3%82%B3%E3%83%AD%E3%83%8A%E7%9B%B8%E5%A0%B4%E3%82%92%E4%B9%97%E3%82%8A%E5%88%87%E3%82%8B%E6%8A%80%E3%80%8D-%E7%AC%AC2%E5%9B%9E%20%7C%20%E7%89%B9%E9%9B%86%20-%20%E6%A0%AA%E6%8E%A2%E3%83%8B%E3%83%A5%E3%83%BC%E3%82%B9)

In [ ]:

import numpy as np

# Import Pipeline class and datasets
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import USEquityPricing

from quantopian.pipeline.data.morningstar import Fundamentals

from quantopian.pipeline.data.factset import Fundamentals as factst_funda
from quantopian.pipeline.filters import Q500US, StaticSids, StaticAssets

# Import built-in trading universe
from quantopian.pipeline.filters import QTradableStocksUS


def make_pipeline():
    # Create a reference to our trading universe
    base_universe = QTradableStocksUS()

    # Get latest closing price
    close_price = USEquityPricing.close.latest
    open_price = USEquityPricing.open.latest
    high_price = USEquityPricing.high.latest
    low_price = USEquityPricing.low.latest
    vol = USEquityPricing.volume.latest
    

    
    #current_liabilities
    #The debts or obligations of the firm that are due within one year.
    current_liabilities = Fundamentals.current_liabilities.latest
    
    #total_assets
    #The aggregate amount of probable future economic benefits obtained or controlled by a particular enterprise as a result of past transactions or events.
    total_assets = Fundamentals.total_assets.latest
    market_cap = Fundamentals.market_cap.latest
    morningstar_industry_group_code = Fundamentals.morningstar_industry_group_code.latest
    morningstar_industry_code = Fundamentals.morningstar_industry_code.latest
    total_liabilities_net_minority_interest = Fundamentals.total_liabilities_net_minority_interest.latest
    operating_income = Fundamentals.operating_income.latest
    interest_expense = Fundamentals.interest_expense.latest
    
    # interest_expense

    myassets = StaticAssets(symbols(["SPY",]))#'AAPL', 'FB', 'MSFT', 'AMZN', 'GOOG', 'KO'
    
    # Return Pipeline containing close_price and
    # sentiment_score that has our trading universe as screen
    return Pipeline(
        columns={
            'close_price': close_price,
            'open_price': open_price, 
            'high_price': high_price, 
            'low_price': low_price, 
            'vol':vol,
            
        },
        screen=myassets #base_universe #myassets
    )

# Import run_pipeline method
from quantopian.research import run_pipeline

# Execute pipeline created by make_pipeline
# between start_date and end_date
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date='2020-01-01',
    end_date='2020-04-17'
)

# Display last 10 rows
pipeline_output.tail(10)

In [ ]:
spy = pipeline_output.xs(symbols('SPY'), level=1)
# 前日より出来高を増加させて、かつ前日より1.25％以上、上昇する」
spy["volchange"] = spy["vol"].pct_change() > 0
spy["prev_change"] = spy["close_price"].pct_change() 
spy["plus1.25"] = spy["prev_change"] > 0.0125
spy["_qriteria3"] = spy["volchange"] & spy["plus1.25"]

n = 4
spy["qriteria3"] = spy["_qriteria3"].rolling(window=n).sum() >= 1


# 前日の low よりも今日の close が高い
spy["close_larger_than_prev_low"] = spy["close_price"] > spy["low_price"].shift(1) 

# 「Day1」を起点として、2日目「Day 2」と3日目「Day 3」がDay1のザラ場の安値を下回らないこと 
win = 2
spy["qriteria2"] = spy["close_larger_than_prev_low"].shift(n).rolling(window=win).sum() >=win
spy["qriteria1"] = spy["prev_change"].shift(n+win+1) > 0 
spy["follow_thru"] = spy["qriteria1"] & spy["qriteria2"] & spy["qriteria3"] 

spy["sokone?"] = spy["follow_thru"].shift((n+win+1)*-1)
spy["sokone?"] = spy["sokone?"].replace(np.nan,False)


In [ ]:
spy[["close_price", "sokone?"]].plot(subplots=True)